# NS delayes notification

NS API address: https://www.ns.nl/en/travel-information/ns-api

credentials management: https://dev.to/jamestimmins/django-cheat-sheet-keep-credentials-secure-with-environment-variables-2ah5
https://pypi.org/project/python-dotenv/


In [337]:
#!pip install python-dotenv

In [436]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, pandas as pd
import json
import os
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 300) # display 300 columns without shrinking
pd.set_option('display.max_rows', 100) # Show more rows

In [481]:
from dotenv import load_dotenv
load_dotenv(override=True)
# from pathlib import Path  # python3 only using specific path
# env_path = Path('.') / '.env'
# load_dotenv(dotenv_path=env_path)

True

#Set UIC code, stations code can be googled with universal destinations.
denbosch = 8400319
amszuid = 8400061 

In [357]:
# all ride information

key = os.getenv("NS_KEY")

headers = {
    # Request headers
    'Accept': '',
    'X-Request-ID': '',
    'X-Caller-ID': '',
    'x-api-key': '',
    'Authorization': '',
    'Ocp-Apim-Subscription-Key': '%s' % key,
} 

params = urllib.parse.urlencode({
    # Request parameters
    'originUicCode': '8400061',
    'destinationUicCode': '8400319',
    'previousAdvices': 2,
    'nextAdvices': 8 ,
})

try:
    conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
    conn.request("GET", "/public-reisinformatie/api/v3/trips?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    #print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [358]:
d = json.loads(data)

In [359]:
#d

In [360]:
#df = json_normalize(d['payload'])
d_trips = json_normalize(d['trips'])

In [361]:
#type(d_trips)

In [362]:
df = d_trips.query('transfers == 0')

In [363]:
df = df.reset_index()

In [364]:
df_lcopy = df['legs'].copy()

In [365]:
df_lcopy

0    [{'idx': '0', 'name': 'IC 3521 ', 'travelType'...
1    [{'idx': '0', 'name': 'IC 3523 ', 'travelType'...
2    [{'idx': '0', 'name': 'IC 3525 ', 'travelType'...
Name: legs, dtype: object

In [366]:
frames = []
for i in range(len(df_lcopy)):    
    frames.append(json_normalize(df_lcopy[i]))

In [367]:
df_leg = pd.concat(frames, sort='True')

In [368]:
df_leg

,alternativeTransport,cancelled,changePossible,crowdForecast,destination.checkinStatus,destination.countryCode,destination.exitSide,destination.lat,destination.lng,destination.name,destination.notes,destination.plannedDateTime,destination.plannedTimeZoneOffset,destination.plannedTrack,destination.products,destination.prognosisType,destination.type,destination.uicCode,destination.weight,direction,idx,journeyDetail,journeyDetailRef,messages,name,notes,origin.checkinStatus,origin.countryCode,origin.lat,origin.lng,origin.name,origin.notes,origin.plannedDateTime,origin.plannedTimeZoneOffset,origin.plannedTrack,origin.products,origin.prognosisType,origin.type,origin.uicCode,origin.weight,product.categoryCode,product.displayName,product.longCategoryName,product.number,product.operatorCode,product.operatorName,product.shortCategoryName,product.type,punctuality,reachable,shorterStock,steps,stops,travelType
0,False,False,True,NaN,CHECKOUT,NL,NaN,51.690536,5.29479,Hertogenbosch ('s),[{'value': 'Station Accessible or Travel Assis...,2019-12-09T07:52:00+0100,60,6,-1,PROGNOSED,STATION,8400319,-1,Venlo,0,"[{'type': 'TRAIN_XML', 'link': {'uri': 'https:...",1|1221|0|784|9122019,[],IC 3521,"[{'value': '+eas, source RAST attributonstatio...",CHECKIN,NL,52.339027,4.873061,Amsterdam Zuid,[{'value': 'Station Accessible or Travel Assis...,2019-12-09T06:56:00+0100,60,2,-1,PROGNOSED,STATION,8400061,-1,IC,NS Intercity,Intercity,3521,ns,NS,IC,TRAIN,100.0,False,False,[],[{'notes': [{'value': 'Station Accessible or T...,PUBLIC_TRANSIT
0,False,False,True,MEDIUM,CHECKOUT,NL,RIGHT,51.690536,5.29479,Hertogenbosch ('s),[{'value': 'Station Accessible or Travel Assis...,2019-12-09T08:22:00+0100,60,6,-1,PROGNOSED,STATION,8400319,-1,Venlo,0,"[{'type': 'TRAIN_XML', 'link': {'uri': 'https:...",1|1221|1|784|9122019,[],IC 3523,"[{'value': '+eas, source RAST attributonstatio...",CHECKIN,NL,52.339027,4.873061,Amsterdam Zuid,[{'value': 'Station Accessible or Travel Assis...,2019-12-09T07:26:00+0100,60,2,-1,PROGNOSED,STATION,8400061,-1,IC,NS Intercity,Intercity,3523,ns,NS,IC,TRAIN,83.3,False,False,[],[{'notes': [{'value': 'Station Accessible or T...,PUBLIC_TRANSIT
0,False,False,True,NaN,CHECKOUT,NL,RIGHT,51.690536,5.29479,Hertogenbosch ('s),[{'value': 'Station Accessible or Travel Assis...,2019-12-09T08:52:00+0100,60,6,-1,PROGNOSED,STATION,8400319,-1,Venlo,0,"[{'type': 'TRAIN_XML', 'link': {'uri': 'https:...",1|442297|0|784|9122019,[],IC 3525,"[{'value': '+eas, source RAST attributonstatio...",CHECKIN,NL,52.339027,4.873061,Amsterdam Zuid,[{'value': 'Station Accessible or Travel Assis...,2019-12-09T07:56:00+0100,60,2,-1,PROGNOSED,STATION,8400061,-1,IC,NS Intercity,Intercity,3525,ns,NS,IC,TRAIN,83.3,False,False,[],[{'notes': [{'value': 'Station Accessible or T...,PUBLIC_TRANSIT


In [383]:
#df_leg['destination.notes'].values
#df_leg['notes'].values

In [483]:
columns_select = ['destination.prognosisType', 'origin.name','destination.name', 'cancelled','origin.plannedDateTime', 'origin.actualDateTime', 'destination.plannedDateTime','punctuality']
all_columns = df_leg.columns

In [484]:
intersection = set(columns_select) & set(all_columns)

In [486]:
df_an = df_leg[intersection]

In [487]:
df_an

,destination.prognosisType,punctuality,cancelled,destination.plannedDateTime,origin.name,destination.name,origin.plannedDateTime
0,PROGNOSED,100.0,False,2019-12-09T07:52:00+0100,Amsterdam Zuid,Hertogenbosch ('s),2019-12-09T06:56:00+0100
0,PROGNOSED,83.3,False,2019-12-09T08:22:00+0100,Amsterdam Zuid,Hertogenbosch ('s),2019-12-09T07:26:00+0100
0,PROGNOSED,83.3,False,2019-12-09T08:52:00+0100,Amsterdam Zuid,Hertogenbosch ('s),2019-12-09T07:56:00+0100


In [454]:
import smtplib

gmail_user = os.getenv('GMAIL_USER')
gmail_password = os.getenv('GMAIL_PWD')

# try:
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.ehlo()
server.login(gmail_user, gmail_password)
# except:
#     print('Something went wrong...')

(235, b'2.7.0 Accepted')

In [ ]:
import smtplib

gmail_user = os.getenv('GMAIL_USER')
gmail_password = os.getenv('GMAIL_PWD')
send_to = list(os.getenv("SEND_TO").split(";")) 

sent_from = gmail_user
to = send_to
subject = 'Test message'
body = 'NS status' + df_an

email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, email_text)
    server.close()

    print 'Email sent!'
except:
    print 'Something went wrong...'

In [418]:
def notification():
    message = ''
    if 'origin.actualDateTime' in intersection:
        m = 'Delayed'
        print('Delayed')
    else:
        m = 'OK'
        print('OK')
    return m

In [420]:
notification()
m

OK


NameError: name 'm' is not defined

In [331]:
#dateTime (optional)string
#Format - date-time (as date-time in RFC3339). Departure date / time for the search. defaults to server time (Europe/Amsterdam)

In [23]:
# #disruptions
# headers = {
#     # Request headers
#     'Ocp-Apim-Subscription-Key': '%s' % key ,
# }

# params = urllib.parse.urlencode({
#     # Request parameters
#     'type': '{string}',
#     'actual': '{boolean}',
#     'lang': 'en',
# })

# try:
#     conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
#     conn.request("GET", "/public-reisinformatie/api/v2/disruptions?%s" % params, "{body}", headers)
#     response = conn.getresponse()
#     disruption = response.read()
#     #print(data)
#     conn.close()
# except Exception as e:
#     print("[Errno {0}] {1}".format(e.errno, e.strerror))


b'{"links":{},"payload":[{"id":"6003223","type":"verstoring","titel":"Utrecht-Hilversum; Utrecht-Almere","topic":"disruption_6003223_nl","verstoring":{"type":"STORING","id":"6003223","geldigheidsLijst":[],"verwachting":"De treinen rijden weer normaal rond 20:00 uur.","gevolg":"De vertraging neemt af.","fase":"4","faseLabel":"Storing voorbij","alternatiefVervoer":"","landelijk":false,"oorzaak":"Tussen Utrecht Overvecht en Hilversum zijn de beperkingen voorbij.","header":"Utrecht-Hilversum; Utrecht-Almere","meldtijd":"2019-12-08T18:39:35+0000","baanvakken":[{"stations":["uto","hor","hvsp","hvs"]}],"trajecten":[{"stations":["ut","uto","hor","hvsp","hvs"],"begintijd":"2019-12-08T15:48:11+0000","eindtijd":"2019-12-08T19:09:35+0000","richting":"HEEN_EN_TERUG"},{"stations":["ut","uto","hor","hvsp","hvs","hvsm","bsmz","ndb","ampo","almm","alm"],"begintijd":"2019-12-08T15:48:11+0000","eindtijd":"2019-12-08T19:09:35+0000","richting":"HEEN_EN_TERUG"}],"versie":"1","volgnummer":"6","prioriteit":10

In [92]:
# station = "amsterdamzuid"

In [93]:
# #Disruption station
# headers = {
#     # Request headers
#     'Ocp-Apim-Subscription-Key': '%s' % key ,
# }

# params = urllib.parse.urlencode({
# })

# try:
#     conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
#     conn.request("GET", "/public-reisinformatie/api/v2/disruptions/station/%s" % station, "{body}", headers)
#     response = conn.getresponse()
#     disrupt_station = response.read()
#     #print(data)
#     conn.close()
# except Exception as e:
#     print("[Errno {0}] {1}".format(e.errno, e.strerror))
